# Train a Model with SageMaker Autopilot

We will use Autopilot to predict sentiment of customer reviews. Autopilot implements a unique white-box approach to AutoML. 

For more on Autopilot, you can review the following research paper:  https://assets.amazon.science/e8/8b/2366b1ab407990dec96e55ee5664/amazon-sagemaker-autopilot-a-white-box-automl-solution-at-scale.pdf

<img src="img/autopilot.png" width="80%" align="left">

# Introduction

Amazon SageMaker Autopilot is a service to perform automated machine learning (AutoML) on your datasets.  Autopilot is available through the UI or AWS SDK.  In this notebook, we will use the AWS SDK to create and deploy a text processing and sentiment classification machine learning pipeline.

# Pre-Requisite

## Make sure the previous notebook has run fully and prepared the dataset.

# Setup

Let's start by specifying:

* The S3 bucket and prefix to use to train our model.  _Note:  This should be in the same region as this notebook._
* The IAM role of this notebook needs access to your data.

# Note:  This notebook will take some time.  Feel free to continue to the next notebooks whenever you are waiting for the current notebook to finish.
We do this throughout the entire workshop as some of these notebooks may run for a while.

In [2]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

# Dataset

In [3]:
%store -r header_train_s3_uri

In [4]:
if not header_train_s3_uri:
    print('****************************************************************************************')
    print('**************** PLEASE RE-RUN THE PREVIOUS DATA PREPARATION NOTEBOOK ******************')
    print('**************** THIS NOTEBOOK WILL NOT RUN PROPERLY ***********************************')
    print('****************************************************************************************')

In [5]:
print(header_train_s3_uri)

s3://sagemaker-us-west-2-250107111215/data/amazon_reviews_us_Digital_Software_v1_00_header.csv


In [6]:
!aws s3 ls $header_train_s3_uri

2020-08-22 16:05:53   13579830 amazon_reviews_us_Digital_Software_v1_00_header.csv


## See our prepared training data which we use as input for Autopilot

In [7]:
!aws s3 cp $header_train_s3_uri ./data/

download: s3://sagemaker-us-west-2-250107111215/data/amazon_reviews_us_Digital_Software_v1_00_header.csv to data/amazon_reviews_us_Digital_Software_v1_00_header.csv


In [8]:
import csv

df = pd.read_csv('./data/amazon_reviews_us_Digital_Software_v1_00_header.csv')
df.head()

,star_rating,review_body
0,1,"I purchased 2 Intuit produces, QuickBooks and ..."
1,3,This is a great map program but can be very co...
2,4,Taxing but worth it
3,5,Okay
4,4,No issues downloading as some reviewers noted....


# Setup the S3 Location for the Autopilot-Generated Assets 
This include Jupyter Notebooks (Analysis), Python Scripts (Feature Engineering), and Trained Models.

In [9]:
prefix_model_output = 'models/autopilot'

model_output_s3_uri = 's3://{}/{}'.format(bucket, prefix_model_output)

print(model_output_s3_uri)


s3://sagemaker-us-west-2-250107111215/models/autopilot


In [10]:
max_candidates = 3

job_config = {
    'CompletionCriteria': {
      'MaxRuntimePerTrainingJobInSeconds': 600,
      'MaxCandidates': max_candidates,
      'MaxAutoMLJobRuntimeInSeconds': 3600
    },
}

input_data_config = [{
      'DataSource': {
        'S3DataSource': {
          'S3DataType': 'S3Prefix',
          'S3Uri': '{}'.format(header_train_s3_uri)
        }
      },
      'TargetAttributeName': 'star_rating'
    }
]

output_data_config = {
    'S3OutputPath': '{}'.format(model_output_s3_uri)
}

# Launch the SageMaker Autopilot job

We can now launch the job by calling the `create_auto_ml_job` API.

In [11]:
from time import gmtime, strftime, sleep
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())

auto_ml_job_name = 'automl-dm-' + timestamp_suffix
print('AutoMLJobName: ' + auto_ml_job_name)

AutoMLJobName: automl-dm-22-16-24-40


_Note that we are not specifying the `ProblemType`.  Autopilot will automatically detect if we're using regression or classification (binary or multi-class)._

In [12]:
sm.create_auto_ml_job(AutoMLJobName=auto_ml_job_name,
                      InputDataConfig=input_data_config,
                      OutputDataConfig=output_data_config,
                      AutoMLJobConfig=job_config,
                      RoleArn=role)

{'AutoMLJobArn': 'arn:aws:sagemaker:us-west-2:250107111215:automl-job/automl-dm-22-16-24-40',
 'ResponseMetadata': {'RequestId': '845c2669-4f57-4e71-b6e3-e05f2cd3ea04',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '845c2669-4f57-4e71-b6e3-e05f2cd3ea04',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '92',
   'date': 'Sat, 22 Aug 2020 16:25:22 GMT'},
  'RetryAttempts': 0}}

# Tracking the progress of the Autopilot job

SageMaker Autopilot job consists of the following high-level steps: 
* _Data Analysis_ where the data is summarized and analyzed to determine which feature engineering techniques, hyper-parameters, and models to explore.
* _Feature Engineering_ where the data is scrubbed, balanced, combined, and split into train and validation.
* _Model Training and Tuning_ where the top performing features, hyper-parameters, and models are selected and trained.

# Analyzing Data

In [13]:
# Sleep for a bit to ensure the AutoML job above has time to start
import time
time.sleep(30)

job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
print(job)

{'AutoMLJobName': 'automl-dm-22-16-24-40', 'AutoMLJobArn': 'arn:aws:sagemaker:us-west-2:250107111215:automl-job/automl-dm-22-16-24-40', 'InputDataConfig': [{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-west-2-250107111215/data/amazon_reviews_us_Digital_Software_v1_00_header.csv'}}, 'TargetAttributeName': 'star_rating'}], 'OutputDataConfig': {'S3OutputPath': 's3://sagemaker-us-west-2-250107111215/models/autopilot'}, 'RoleArn': 'arn:aws:iam::250107111215:role/TeamRole', 'AutoMLJobConfig': {'CompletionCriteria': {'MaxCandidates': 3, 'MaxRuntimePerTrainingJobInSeconds': 600, 'MaxAutoMLJobRuntimeInSeconds': 3600}}, 'CreationTime': datetime.datetime(2020, 8, 22, 16, 25, 21, 488000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2020, 8, 22, 16, 27, 9, 510000, tzinfo=tzlocal()), 'AutoMLJobStatus': 'InProgress', 'AutoMLJobSecondaryStatus': 'AnalyzingData', 'GenerateCandidateDefinitionsOnly': False, 'ResponseMetadata': {'RequestId': '11da101b-2

### Check if the Autopilot Job started correctly.

In [14]:
if not bool(job):
    print('STOP: Autopilot Job did NOT start correctly. Please re-run the notebook from start.')
elif 'AutoMLJobStatus' not in job.keys():
    print('STOP: Autopilot Job did NOT start correctly. Please re-run the notebook from start.')
elif 'AutoMLJobSecondaryStatus' not in job.keys():
    print('STOP: Autopilot Job did NOT start correctly. Please re-run the notebook from start.')
else:
    print('OK')

OK


In [15]:
job_status = job['AutoMLJobStatus']
job_sec_status = job['AutoMLJobSecondaryStatus']

if job_status not in ('Stopped', 'Failed'):
    while job_status in ('InProgress') and job_sec_status in ('AnalyzingData'):
        job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
        job_status = job['AutoMLJobStatus']
        job_sec_status = job['AutoMLJobSecondaryStatus']
        print(job_status, job_sec_status)
        sleep(30)
    print("Data analysis complete")
    
print(job)

InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress FeatureEngineering
Data analysis complete
{'AutoMLJobName': 'automl-dm-22-16-24-40', 'AutoMLJobArn': 'arn:aws:sagemaker:us-west-2:250107111215:automl-job/automl-dm-22-16-24-40', 'InputDataConfig': [{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-west-2-250107111215/data/amazon_reviews_us_Digital_Software_v1_00_header.csv'}}, 'TargetAttributeName': 'star_rating'}], 'OutputDataConfig': {'S3OutputPath': 's3://sagemaker-us-west-2-250107111215/models/autopilot'}, 'RoleArn': 'arn:aws:iam::250107111215:role/TeamRole', 'AutoMLJobConfig': {'CompletionCriteria': {'MaxCandidates': 3, 'MaxRuntimePerTrainingJobInSeconds': 600, 'MaxAutoMLJobRuntimeInSeconds': 3600}}, 'CreationTime

In [16]:
job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
print(job)

{'AutoMLJobName': 'automl-dm-22-16-24-40', 'AutoMLJobArn': 'arn:aws:sagemaker:us-west-2:250107111215:automl-job/automl-dm-22-16-24-40', 'InputDataConfig': [{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-west-2-250107111215/data/amazon_reviews_us_Digital_Software_v1_00_header.csv'}}, 'TargetAttributeName': 'star_rating'}], 'OutputDataConfig': {'S3OutputPath': 's3://sagemaker-us-west-2-250107111215/models/autopilot'}, 'RoleArn': 'arn:aws:iam::250107111215:role/TeamRole', 'AutoMLJobConfig': {'CompletionCriteria': {'MaxCandidates': 3, 'MaxRuntimePerTrainingJobInSeconds': 600, 'MaxAutoMLJobRuntimeInSeconds': 3600}}, 'CreationTime': datetime.datetime(2020, 8, 22, 16, 25, 21, 488000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2020, 8, 22, 16, 34, 25, 639000, tzinfo=tzlocal()), 'AutoMLJobStatus': 'InProgress', 'AutoMLJobSecondaryStatus': 'FeatureEngineering', 'GenerateCandidateDefinitionsOnly': False, 'AutoMLJobArtifacts': {'CandidateDefini

# View Generated Notebook Samples
Once data analysis is complete, SageMaker AutoPilot generates two notebooks: 
* Data exploration,
* Candidate definition.

# In the Jupyter File Browser, Open the Following Folders to See Samples of the Generated Assets:
```
notebooks/
generated_module/
```

Lots of useful information ^^ in these folders ^^

(Optional) You can download the actual files generated for your specific Autopilot run using the following:
```
generated_resources = job['AutoMLJobArtifacts']['DataExplorationNotebookLocation'].rstrip('notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb')

!aws s3 cp --recursive $generated_resources .
```

# Feature Engineering

In [17]:
job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
job_status = job['AutoMLJobStatus']
job_sec_status = job['AutoMLJobSecondaryStatus']
print(job_status)
print(job_sec_status)
if job_status not in ('Stopped', 'Failed'):
    while job_status in ('InProgress') and job_sec_status in ('FeatureEngineering'):
        job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
        job_status = job['AutoMLJobStatus']
        job_sec_status = job['AutoMLJobSecondaryStatus']
        print(job_status, job_sec_status)
        sleep(30)
    print("Feature engineering complete")
    
print(job)

InProgress
FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress ModelTuning
Feature engineering complete
{'AutoMLJobName': 'automl-dm-22-16-24-40', 'AutoMLJobArn': 'arn:aws:sagemaker:us-west-2:250107111215:automl-job/automl-dm-22-16-24-40', 'InputDataConfig': [{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-west-2-250107111215/data/amazon_reviews_us_Digital_Software_v1_00_header.csv'}}, 'TargetAttributeName': 'star_rating'}], 'OutputDataConfig': {'S3OutputPath': 's3://sagemaker-us-west-2-250107111215/models/autopilot'}, 'RoleArn

# Model Training and Tuning

In [18]:
job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
job_status = job['AutoMLJobStatus']
job_sec_status = job['AutoMLJobSecondaryStatus']
print(job_status)
print(job_sec_status)
if job_status not in ('Stopped', 'Failed'):
    while job_status in ('InProgress') and job_sec_status in ('ModelTuning'):
        job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
        job_status = job['AutoMLJobStatus']
        job_sec_status = job['AutoMLJobSecondaryStatus']
        print(job_status, job_sec_status)
        sleep(30)
    print("Model tuning complete")
    
print(job)

InProgress
ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
Completed MaxCandidatesReached
Model tuning complete
{'AutoMLJobName': 'automl-dm-22-16-24-40', 'AutoMLJobArn': 'arn:aws:sagemaker:us-west-2:250107111215:automl-job/automl-dm-22-16-24-40', 'InputDataConfig': [{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-west-2-250107111215/data/amazon_reviews_us_Digital_Software_v1_00_header.csv'}}, 'TargetAttributeName': 'star_rating'}], 'OutputDataConfig': {'S3OutputPath': 's3://sagemaker-us-west-2-250107111215/models/autopilot'}, 'RoleArn': 'arn:aws:iam::250107111215:role/TeamRole', 'AutoMLJobConfig': {'CompletionCriteria': {'MaxCandidates': 3, 'MaxRuntimePerTrainingJobInSeconds': 600, 'MaxAutoMLJobRuntimeInSeconds': 3600}}, 'CreationTime': datetime.datetime(2020, 8, 22, 16, 25, 21, 488000, tzinfo=tzlocal()), 'EndTime': datetim

# _Please Wait Until ^^ Autopilot ^^ Completes Above_
Make sure the status below indicates `Completed`.

In [19]:
print(job_status)

if job_status not in ('Completed'):
    print('*******************************************************************')
    print('*************** THIS JOB DID NOT COMPLETE PROPERLY ****************')
    print('***************  REPORT THE ISSUE OR ASK FOR HELP  ****************')    
    print('*******************************************************************')

Completed


# Viewing All Candidates
Once model tuning is complete, you can view all the candidates (pipeline evaluations with different hyperparameter combinations) that were explored by AutoML and sort them by their final performance metric.

In [20]:
candidates_response = sm.list_candidates_for_auto_ml_job(AutoMLJobName=auto_ml_job_name, 
                                                         SortBy='FinalObjectiveMetricValue')

### Check that candidates is not empty

In [21]:
if not bool(candidates_response):
    print('STOP: Autopilot Job did NOT finish correctly. Please re-run the notebook from start.')
else:
    candidates = candidates_response['Candidates']
    print('OK')

OK


In [22]:
if not bool(candidates):
    print('STOP: Autopilot Job did NOT finish correctly. Please re-run the notebook from start.')
elif 'CandidateName' not in candidates[0]:
    print('STOP: Autopilot Job did NOT finish correctly. Please re-run the notebook from start.')
elif 'FinalAutoMLJobObjectiveMetric' not in candidates[0]:
    print('STOP: Autopilot Job did NOT start correctly. Please re-run the notebook from start.')
else:
    print('OK')

OK


In [23]:
for index, candidate in enumerate(candidates):
    print(str(index) + "  " 
        + candidate['CandidateName'] + "  " 
        + str(candidate['FinalAutoMLJobObjectiveMetric']['Value']))

0  tuning-job-1-eece15bc5b8b4ed8aa-003-1187ffc4  0.39945000410079956
1  tuning-job-1-eece15bc5b8b4ed8aa-001-2684dc0d  0.3988099992275238
2  tuning-job-1-eece15bc5b8b4ed8aa-002-4dfa8be1  0.2757599949836731


# Inspect Trials using Experiments API

SageMaker Autopilot automatically creates a new experiment, and pushes information for each trial. 

In [24]:
from sagemaker.analytics import ExperimentAnalytics, TrainingJobAnalytics

exp = ExperimentAnalytics(
    sagemaker_session=sess, 
    experiment_name=auto_ml_job_name + '-aws-auto-ml-job',
)

df = exp.dataframe()
print(df)

                                  TrialComponentName  \
0  tuning-job-1-eece15bc5b8b4ed8aa-003-1187ffc4-a...   
1  tuning-job-1-eece15bc5b8b4ed8aa-001-2684dc0d-a...   
2  tuning-job-1-eece15bc5b8b4ed8aa-002-4dfa8be1-a...   
3  automl-dm--dpp2-rpb-1-9740205c750f4253b1bb300b...   
4  automl-dm--dpp1-csv-1-92e5fe206f514430919b6797...   
5  automl-dm--dpp0-rpb-1-e7fd271f2f7b4834b49a1779...   
6  automl-dm--dpp2-1-b4427efb8653498f921496864131...   
7  automl-dm--dpp0-1-d241ad12c8944d64bc7be538a4e9...   
8  automl-dm--dpp1-1-95dc69438ac9483a868066c76e9a...   
9  db-1-74124d35b29d4000bd9de8426901cad8006d23d4a...   

                                         DisplayName  \
0  tuning-job-1-eece15bc5b8b4ed8aa-003-1187ffc4-a...   
1  tuning-job-1-eece15bc5b8b4ed8aa-001-2684dc0d-a...   
2  tuning-job-1-eece15bc5b8b4ed8aa-002-4dfa8be1-a...   
3  automl-dm--dpp2-rpb-1-9740205c750f4253b1bb300b...   
4  automl-dm--dpp1-csv-1-92e5fe206f514430919b6797...   
5  automl-dm--dpp0-rpb-1-e7fd271f2f7b4834b49a17

# Explore the Best Candidate
Now that we have successfully completed the AutoML job on our dataset and visualized the trials, we can create a model from any of the trials with a single API call and then deploy that model for online or batch prediction using [Inference Pipelines](https://docs.aws.amazon.com/sagemaker/latest/dg/inference-pipelines.html). For this notebook, we deploy only the best performing trial for inference.

The best candidate is the one we're really interested in.

In [25]:
best_candidate_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)

In [26]:
if not bool(best_candidate_response):
    print('STOP: Autopilot Job did NOT finish correctly. Please re-run the notebook from start.')
else:
    best_candidate = best_candidate_response['BestCandidate']
    print('OK')

OK


In [27]:
if not bool(best_candidate):
    print('STOP: Autopilot Job did NOT finish correctly. Please re-run the notebook from start.')
elif 'CandidateName' not in best_candidate:
    print('STOP: Autopilot Job did NOT finish correctly. Please re-run the notebook from start.')
elif 'FinalAutoMLJobObjectiveMetric' not in best_candidate:
    print('STOP: Autopilot Job did NOT finish correctly. Please re-run the notebook from start.')
else:
    best_candidate_identifier = best_candidate['CandidateName']
    print("Candidate name: " + best_candidate_identifier)
    print("Metric name: " + best_candidate['FinalAutoMLJobObjectiveMetric']['MetricName'])
    print("Metric value: " + str(best_candidate['FinalAutoMLJobObjectiveMetric']['Value']))


Candidate name: tuning-job-1-eece15bc5b8b4ed8aa-003-1187ffc4
Metric name: validation:accuracy
Metric value: 0.39945000410079956


In [28]:
best_candidate

{'CandidateName': 'tuning-job-1-eece15bc5b8b4ed8aa-003-1187ffc4',
 'FinalAutoMLJobObjectiveMetric': {'MetricName': 'validation:accuracy',
  'Value': 0.39945000410079956},
 'ObjectiveStatus': 'Succeeded',
 'CandidateSteps': [{'CandidateStepType': 'AWS::SageMaker::ProcessingJob',
   'CandidateStepArn': 'arn:aws:sagemaker:us-west-2:250107111215:processing-job/db-1-74124d35b29d4000bd9de8426901cad8006d23d4ab574a2a9bd18f9c64',
   'CandidateStepName': 'db-1-74124d35b29d4000bd9de8426901cad8006d23d4ab574a2a9bd18f9c64'},
  {'CandidateStepType': 'AWS::SageMaker::TrainingJob',
   'CandidateStepArn': 'arn:aws:sagemaker:us-west-2:250107111215:training-job/automl-dm--dpp0-1-d241ad12c8944d64bc7be538a4e91c120aad50095dc94',
   'CandidateStepName': 'automl-dm--dpp0-1-d241ad12c8944d64bc7be538a4e91c120aad50095dc94'},
  {'CandidateStepType': 'AWS::SageMaker::TransformJob',
   'CandidateStepArn': 'arn:aws:sagemaker:us-west-2:250107111215:transform-job/automl-dm--dpp0-rpb-1-e7fd271f2f7b4834b49a177902adc1484bd

We can see the containers and models composing the Inference Pipeline.

In [29]:
if not bool(best_candidate):
    print('STOP: Autopilot Job did NOT finish correctly. Please re-run the notebook from start.')
elif 'InferenceContainers' not in best_candidate:
    print('STOP: Autopilot Job did NOT finish correctly. Please re-run the notebook from start.')
else:
    for container in best_candidate['InferenceContainers']:
        print(container['Image'])
        print(container['ModelDataUrl'])
        print('======================')


246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-sklearn-automl:0.1.0-cpu-py3
s3://sagemaker-us-west-2-250107111215/models/autopilot/automl-dm-22-16-24-40/data-processor-models/automl-dm--dpp0-1-d241ad12c8944d64bc7be538a4e91c120aad50095dc94/output/model.tar.gz
246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3
s3://sagemaker-us-west-2-250107111215/models/autopilot/automl-dm-22-16-24-40/tuning/automl-dm--dpp0-xgb/tuning-job-1-eece15bc5b8b4ed8aa-003-1187ffc4/output/model.tar.gz
246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-sklearn-automl:0.1.0-cpu-py3
s3://sagemaker-us-west-2-250107111215/models/autopilot/automl-dm-22-16-24-40/data-processor-models/automl-dm--dpp0-1-d241ad12c8944d64bc7be538a4e91c120aad50095dc94/output/model.tar.gz


# Autopilot Chooses XGBoost as Best Candidate!

Note that Autopilot chose different hyper-parameters and feature transformations than we used in our own XGBoost model.

# Deploy the Model as a REST Endpoint
Batch transformations are also supported, but for now, we will use a REST Endpoint.

In [30]:
model_name = 'automl-dm-model-' + timestamp_suffix

model_arn = sm.create_model(Containers=best_candidate['InferenceContainers'],
                            ModelName=model_name,
                            ExecutionRoleArn=role)

print('Best candidate model ARN: ', model_arn['ModelArn'])

Best candidate model ARN:  arn:aws:sagemaker:us-west-2:250107111215:model/automl-dm-model-22-16-24-40


In [31]:
# EndpointConfig name
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())
epc_name = 'automl-dm-epc-' + timestamp_suffix

# Endpoint name
autopilot_endpoint_name = 'automl-dm-ep-' + timestamp_suffix
variant_name = 'automl-dm-variant-' + timestamp_suffix

print(autopilot_endpoint_name)
print(variant_name)

automl-dm-ep-22-16-47-12
automl-dm-variant-22-16-47-12


In [32]:
ep_config = sm.create_endpoint_config(EndpointConfigName = epc_name,
                                      ProductionVariants=[{'InstanceType':'ml.m5.large',
                                                           'InitialInstanceCount': 1,
                                                           'ModelName': model_name,
                                                           'VariantName': variant_name}])


In [33]:
create_endpoint_response = sm.create_endpoint(EndpointName=autopilot_endpoint_name,
                                              EndpointConfigName=epc_name)
print(create_endpoint_response['EndpointArn'])

arn:aws:sagemaker:us-west-2:250107111215:endpoint/automl-dm-ep-22-16-47-12


# Wait for the Model to Deploy
This may take 5-10 mins.  Please be patient.

In [34]:
sm.get_waiter('endpoint_in_service').wait(EndpointName=autopilot_endpoint_name)


In [35]:
resp = sm.describe_endpoint(EndpointName=autopilot_endpoint_name)
status = resp['EndpointStatus']

print("Arn: " + resp['EndpointArn'])
print("Status: " + status)

Arn: arn:aws:sagemaker:us-west-2:250107111215:endpoint/automl-dm-ep-22-16-47-12
Status: InService


# Test Our Model with Some Example Reviews
Let's do some ad-hoc predictions on our model.

In [36]:
sm_runtime = boto3.client('sagemaker-runtime')

In [37]:
csv_line_predict_positive = """I loved it!"""

response = sm_runtime.invoke_endpoint(EndpointName=autopilot_endpoint_name, ContentType='text/csv', Accept='text/csv', Body=csv_line_predict_positive)

response_body = response['Body'].read().decode('utf-8').strip()
response_body

'5'

In [38]:
csv_line_predict_meh = """It's OK."""

response = sm_runtime.invoke_endpoint(EndpointName=autopilot_endpoint_name, ContentType='text/csv', Accept='text/csv', Body=csv_line_predict_meh)

response_body = response['Body'].read().decode('utf-8').strip()
response_body

'3'

In [39]:
csv_line_predict_negative = """The worst product ever."""

response = sm_runtime.invoke_endpoint(EndpointName=autopilot_endpoint_name, ContentType='text/csv', Accept='text/csv', Body=csv_line_predict_negative)

response_body = response['Body'].read().decode('utf-8').strip()
response_body

'5'

In [40]:
%store autopilot_endpoint_name

Stored 'autopilot_endpoint_name' (str)


In [41]:
%store

Stored variables and their in-db values:
autopilot_endpoint_name             -> 'automl-dm-ep-22-16-47-12'
header_train_s3_uri                 -> 's3://sagemaker-us-west-2-250107111215/data/amazon
noheader_train_s3_uri               -> 's3://sagemaker-us-west-2-250107111215/data/amazon


In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();

# Summary
We used Autopilot to automatically find the best model, hyper-parameters, and feature-engineering scripts for our dataset.  

Autopilot uses a white-box approach to generate re-usable exploration Jupyter Notebooks and transformation Python scripts to continue to train and deploy our model on new data - well after this initial interaction with the Autopilot service.